# [Step 1] ทำการนำเข้า module ที่จำเป็นต้องใช้ในการทำงานบน Environment (เริ่มต้น)

In [1]:
import os, glob, math
import numpy as np
import pyrsgis.raster as raster
from sklearn.utils import resample 

In [ ]:
#ตั้งค่าที่อยู่ของ Folder หรือ Directory ต้นทางเพื่อนำเข้า input เข้ามาใช้งาน
os.chdir(r"C:\Users\std\Desktop\Dataset\CNN Method\SplitIMG_3x3\Output")

#เมื่อ Directory ข้อมูลแล้ว จะทำในขั้นตอนของการตรวจสอบข้อมูลที่ได้
nChipimages = len(glob.glob('*.tif'))

ds, ImagesArr = raster.read(os.listdir(os.getcwd())[0])
nBands, X_Size, Y_Size = ds.RasterCount, ds.RasterXSize, ds.RasterYSize

#สร้างที่เก็บข้อมูลภาพถ่ายและ LU เพื่อที่จะจัดเก็บข้อมูลให้อยู่ในรูปแบบของ Array file (.npy)
Images_IMG = np.empty((nChipimages, nBands, X_Size, Y_Size))
Labels_IMG = np.empty(nChipimages)
print(f'Check Array file : nBands {nBands} | X_Size {X_Size} | Y_Size {Y_Size}')

In [ ]:
for n, file in enumerate(glob.glob('*.tif')):
    if n == nChipimages:
        break
    if n % 100000 == 0:
        print('Sample Read IMG File: %d of %d' % (n, nChipimages))
    ds, ImagesArr = raster.read(file)
    tempLabel = os.path.splitext(file)[0].split('_')[-1]
    
    Images_IMG[n,:,:,:] = ImagesArr
    Labels_IMG[n] = tempLabel
    
print(f'Data detail : IMG {Images_IMG.shape} | Label {Labels_IMG.shape}')
print(f'Value detail : IMG (max value) {Images_IMG.max()} | Label (max value) {Labels_IMG.max()}')

### [Step 1.1] ทำการ Save data เพื่อให้ง่ายต่อการ Load ข้อมูลและย้ายข้อมูล

In [ ]:
## ตั้งค่าที่อยู่หรือ Directory ที่จัดเก็บข้อมูล
Output_DIR = (r"C:\Users\std\Desktop\Dataset\CNN Method\SplitIMG_5x5\Output")
os.chdir(Output_DIR)

#Save data และจัดเก็บข้อมูล
np.save('CNN_IMG_5x5_20180209.npy', Images_IMG)
np.save('CNN_Label_5x5_20180209.npy', Labels_IMG)
print('Save Location in >>> %s'% os.getcwd())

### [Step 1.2] กรณีที่ต้องการ Balance ข้อมูลแต่ละ Class ให้มีจำนวนทีเท่ากัน

In [ ]:
#จำแนกข้อมูลภาพตามตำแหน่งหรือ Class ที่มีกำหนดไว้
#HKT & LU(Class 1 - 5)
HKT_C1 = HKT_IMG[Label_IMG==1]    #ภาพถ่ายดาวเทียมขนาด 3x3 (Class 1 หรือ Urban)
Lab_C1 = Label_IMG[Label_IMG==1]  

HKT_C2 = HKT_IMG[Label_IMG==2]   #ภาพถ่ายดาวเทียมขนาด 3x3 (Class 2 หรือ Agriculture)
Lab_C2 = Label_IMG[Label_IMG==2]

HKT_C3 = HKT_IMG[Label_IMG==3]   #ภาพถ่ายดาวเทียมขนาด 3x3 (Class 3 หรือ Forest or Green Area)
Lab_C3 = Label_IMG[Label_IMG==3]

HKT_C4 = HKT_IMG[Label_IMG==4]   #ภาพถ่ายดาวเทียมขนาด 3x3 (Class 4 หรือ Water Area)
Lab_C4 = Label_IMG[Label_IMG==4]

HKT_C5 = HKT_IMG[Label_IMG==5]   #ภาพถ่ายดาวเทียมขนาด 3x3 (Class 5 หรือ Bare Land)
Lab_C5 = Label_IMG[Label_IMG==5]

#HKT & LU (Class 0)
HKT_C0 = HKT_IMG[Label_IMG==0]   #ภาพถ่ายดาวเทียมขนาด 3x3 (Class 0 หรือ Background)
Lab_C0 = Label_IMG[Label_IMG==0]

#print for HKT & LU
HKT_Class = [HKT_C1, HKT_C2, HKT_C3, HKT_C4, HKT_C5, HKT_C0]
Label_Class = [Lab_C1, Lab_C2, Lab_C3, Lab_C4, Lab_C5, Lab_C0]
for Class, Value in enumerate(HKT_Class):
    print(f'Class {Class} | number of images IMG {Value.shape}')
print('='*35)
for Class, Value in enumerate(Label_Class):
    print(f'Class {Class} | number of images IMG {Value.shape}')

In [ ]:
#ทำการ Downsample หรือทำการ Balance ข้อมูลภาพที่มีจำนวนมากให้มีจำนวนเท่ากับข้อมูลภาพที่ Class น้อยที่สุด
#Feature data (Downsample)
Feat_C1 = resample(HKT_C1, replace=False, n_samples=HKT_C4.shape[0], random_state=2)
Feat_C2 = resample(HKT_C2, replace=False, n_samples=HKT_C4.shape[0], random_state=2)
Feat_C3 = resample(HKT_C3, replace=False, n_samples=HKT_C4.shape[0], random_state=2)
Feat_C5 = resample(HKT_C5, replace=False, n_samples=HKT_C4.shape[0], random_state=2)
Feat_C0 = resample(HKT_C0, replace=False, n_samples=HKT_C4.shape[0], random_state=2)

#Labels Data (Downsample)
Label_C1 = resample(Lab_C1, replace=False, n_samples=Lab_C4.shape[0], random_state=2)
Label_C2 = resample(Lab_C2, replace=False, n_samples=Lab_C4.shape[0], random_state=2)
Label_C3 = resample(Lab_C3, replace=False, n_samples=Lab_C4.shape[0], random_state=2)
Label_C5 = resample(Lab_C5, replace=False, n_samples=Lab_C4.shape[0], random_state=2)
Label_C0 = resample(Lab_C0, replace=False, n_samples=Lab_C4.shape[0], random_state=2)
# ไม่ทำการ Balance Class 4 เนื่องจากใช้เป็นจำนวนจุดสุ่ม เป็นตัวตั้งต้น

for Cls, val in enumerate([Feat_C1, Feat_C2, Feat_C3, HKT_C4, Feat_C5, Feat_C0]):
    print(f"CLass {Cls} | number of images balance {val.shape}")
print('='*35)
for Cls, val in enumerate([Label_C1, Label_C2, Label_C3, Lab_C4, Label_C5, Label_C0]):
    print(f"CLass {Cls} | number of images balance {val.shape}")

### [Step 1.3] ทำการรวมข้อมูลที่ได้ทำการ Balance Class ให้เป็นข้อมูลตัวเดียวกัน (.npy format)

In [ ]:
#ทำการรวมข้อมูลทีทำการ Downsample Class ทั้งหมดรวมกันเป็น 5 Class + background
HKT_resample = np.concatenate((Feat_C1, Feat_C2, Feat_C3, HKT_C4, Feat_C5, Feat_C0), axis=0)
Label_resample = np.concatenate((Label_C1, Label_C2, Label_C3, Lab_C4, Label_C5, Label_C0), axis=0)
print(f'Detail Data Shape : HKT_IMG = {HKT_resample.shape} | Label_IMG = {Label_resample.shape}')

In [ ]:
#Save The Array file // ทำการ Save ข้อมูลที่ทำการ Downsample Classให้อยู่ในรูปของ Array File
np.save('HKT_CNN_3x3_09022018.npy', HKT_resample)
np.save('Label_CNN_3x3_09022018.npy', Label_resample)